# Exercise 3: Multiple Requests and Callbacks

In this exercise, you will implement inference with multiple requests using callbacks. 

The workload will be once again vehicle detection, but on a video this time. 
Specifically, your application will count the cars in the frame and report three metrics: maximum number of cars in one frame, minimum number of cars in one frame, and average number of cars in all frames.
Run the following cell to see the video.

In [1]:
from IPython.core.display import HTML
HTML("<video alt=\"\" controls autoplay height=\"480\"><source src=\"cars_1900.mp4\" type=\"video/mp4\" /></video>")

### Important! The quiz will ask you the average number of vehicles detected in the last step.


## Implementation

The video course covered some potential implementations using the `wait()` function, including the zero timeout wait.
While the zero timeout example in the video works well, it goes through all the requests over and over again until one of them is done.
In this exercise, you will implement multiple inference that simply waits for the first finished slot using Python queues and inference callbacks

Python queues have a couple of interesting features that make them work well with the multiple request inference workload.
One is that Python queues are thread-safe. 
Without going in to too much detail, this means that the queue is safe to use in an asynchronous setting, like our requests.
The second feature is the get() function (like a "pop" function). If the queue is empty when get() is called, it will wait until an item becomes available. We will begin with an optional section for those who are unfamiliar or need a review of Python Queue.

## (Optional) Step 1: Python queue

This section is designed to give you a brief introduction to Python queue. 
If you are already familiar, skip to step 2.

Python queues are data structures that are accessed in First In First Out (FIFO) order.
When used in an asynchronous workload, this can be used to access the jobs as they complete.

The following is a brief example of using queue in an asynchronous setting.
This example uses threading instead of inference engine, to keep the example simple.
Each thread sleeps for some time, and then puts a tuple containing the id of the thread and how long it slept for.
The main thread will wait on the queue, and print out the contents of the tuple.

In [2]:
import queue
import threading
import time

# Sample asynchronous workload that simply sleeps, and then places ID in queue
def foo(q, myid, timeout):
    time.sleep(timeout)
    q.put((myid, timeout))

# Creating the queue for completed tasks
completion_queue = queue.Queue()

# Create and start two tasks
t1 = threading.Thread(target=foo, args=(completion_queue, 1, 3))
t2 = threading.Thread(target=foo, args=(completion_queue, 2, 1))
t1.start()
t2.start()

# Print tasks as they complete
completed_id, timeout = completion_queue.get()
print("task {} completed after sleeping for {} second(s)".format(completed_id, timeout))
completed_id, timeout = completion_queue.get()
print("task {} completed after sleeping for {} second(s)".format(completed_id, timeout))


# Confirming the threads are completed. Not necessary, but good practice.
t1.join()
t2.join()

task 2 completed after sleeping for 1 second(s)
task 1 completed after sleeping for 3 second(s)


Notice that task 2 had a shorter timeout and completed first. It was printed immediately without waiting for task 1 to complete. Additionally, notice that I did not have to specify any ID in the `get()` function. We will adapt this for inference engine.

## Step 2: Inference Server mock-up

Now we will create a mock-up model of an inference server that can run multiple requests at once.
In exercise-2 we had multiple concurrent requests by starting a number of them and then waiting for all to complete.
However as we discussed in the video, this can be inefficient because some inferences may finish before others.
This is especially true if you are using multiple types of devices.

So to get around this issue, we will set up this server to start inference in a request slot as soon as it is available.
To do this, the server will keep a Python queue that has *available request slots*.
More specifically, each item in the queue contains the ID of the available request slot.
In addition, we will also add the status code of the inference so that the server will know if any request was unsuccessful.

The queue will be populated using the callback function for the request slot. To recap, this is the function that gets called as soon as inference is completed on the request slot. So, we will use this callback functionality to add the ID and the status (as a tuple) of the newly completed request slot.

### utils.py

Begin by writing various helper functions for use in the main loop.
Complete the `utils.py` file by following the instructions.

</br><details>
    <summary><b>(2.1)</b> Complete the <code>prepImage()</code> function by finding the NCHW values from the network.</summary>
    
Complete the `prepImage()` function by getting the values for `n`, `c`, `h` and `w` from the function input `net`.
The code here should be the exact same as in exercise 1.

</details><br/>

<details>
    <summary><b>(2.2)</b> Complete the <code>createExecNetwork()</code> function which takes IECore, IENetwork and device string and returns an ExecutableNetwork with the optimal number of requests.</summary>

To get the optimal number of requests, you first need a default ExecutableNetwork object.
The IENetwork and device string is provided as input argument.
Use these along with IECore to get an ExecutableNetwork.

Then you can get the optimal number of requests from a metric of the ExecutableNetwork. 
See the slides for video 2 of course 2 for more details.
Use this value to recreate an ExecutablkeNetwork object with the optimal number of requests.
Finally, return this executable network.

</details><br/>

<details>
    <summary><b>(2.3)</b> In <code>setCallbackAndQueue()</code> function, add a callback function called <code>callbackFunc</code> to each of the request slots. </summary>
    
We will be defining `callbackFunc` function in step (2.4), but we will work on the part where this callback is added to the request slot.

For our callback function, we need two pieces of information: the ID of the request slot, and the status of the inference.
Additionally, we need access to the queue that keeps track of the completed slots.
The status is automatically made available to the callback function, but the request slot ID as well as access to the queue is not.
So we need to pass these to the function.

To do this, we need to use the `py_data` variable. 
This dictionary variable is set when you add the callback, and is passed in as an argument to the callback function.
For what we need in our callback function, `py_data` must contain the ID of the request slot and the queue.

So first create a dictionary that contains these two. 
The key to use for this dictionary is up to you. 
Then call the  `set_completion_callback()` method for the requests to add the `callbackFunc` (note the lack of parethesis) along with the `py_data`. 
</details><br/>

<details>
    <summary><b>(2.4)</b> Complete <code>callbackFunc()</code> function, by having it add a tuple containing the request slot ID and the status code for the inference.</summary> 

Remember that `py_data` argument is the dictionary you passed in in the previous step.
It should contain the queue and the request ID.
The status of the inference is in the input argument `status`.
Add the tuple (ID, status) to the queue. Note that the order there matters.

</details><br/>

In [3]:
%%writefile utils.py
import cv2
from openvino.inference_engine import IECore, IENetwork

# Prepares image for inference by reshaping and transposing.
# inputs:
#     orig_image - numpy array containing the original, unprocessed image
#     ie_net     - IENetwork object 
def prepImage(input_layer,original_image, ie_net):

    ##! (2.1) Find n, c, h, w from net !##
    n, c, h, w = ie_net.inputs[input_layer].shape

    # Reshaping data    
    input_image = cv2.resize(original_image, (w, h))
    input_image = input_image.transpose((2, 0, 1))
    input_image.reshape((n, c, h, w))

    return input_image

# Processes the result. Returns the number of detected vehices in the image.
# inputs:
#    detected_obects - numpy array containing the ooutput of the model
#    prob_threashold - Required probability for "detection"
# output:
#    Number of vehices detected.
def getCount(detected_objects, prob_threshold=0.5):
    detected_count = 0
    for obj in detected_objects[0][0]:
        # Draw only objects when probability more than specified threshold
        if obj[2] > prob_threshold:
            detected_count+=1
    return detected_count


# Create ExecutableNetwork with the optimal number of requests for a given device.
# inputs:
#    ie_core - IECore object to use
#    ie_net  - IENetwork object to use
#    device  - String to use for device_name argument.
# output:
#    ExecutabeNetwork object
def createExecNetwork(ie_net, device):
    ##! (2.2) Create IECore !##
    ie = IECore()
    ##! (2.2) Create ExecutableNetwork object and find the optimal number of requests !##
    exec_net = ie.load_network(network=ie_net, device_name=device)
    nq = exec_net.get_metric("OPTIMAL_NUMBER_OF_INFER_REQUESTS")
    ##! (2.2) Recreate IECore and with num_requests set to optimal number of requests !##
    exec_net = ie.load_network(network=ie_net, device_name=device, num_requests=nq)
    ##! (2.2) return the ExecutableNetwork !##
    return exec_net

    
# Set callback functions for the inference requests.
# inputs:
#    exec_net - ExecutableNetwork object to modify
#    c_queue  - Python queue to put the slot ID in
def setCallbackAndQueue(exec_net, c_queue):
    for req_slot in range(len(exec_net.requests)):
        ##! (2.3) Create a dictionary for py_data to pass in the queue and ID !###
        data = {"id":req_slot, "queue":c_queue}
        ##! (2.3) Set the completion callback with the arguments for each reqeust !##
        exec_net.requests[req_slot].set_completion_callback(py_callback=callbackFunc, py_data=data)
        # Initializing the queue. The second item of the tuple is the status of the previous 
        #  inference. But as there is no previous inference right now, setting the status to None.
        c_queue.put((req_slot, None))
    
# Callback function called on completion of the inference.
# inputs:
#    status  - status code for the inference.
#    py_data - dictionary arguments passed into the function
def callbackFunc(status, py_data):
    try:
        ##! (2.4) Add a tuple (id, status) to queue here !##
        queue = py_data['queue']
        slot = py_data['id']       
        queue.put((slot, status))
    except:
        print("There was an issue in callback")


Overwriting utils.py


### main.py

Now write the main loop. 
Complete the `main.py` file by following the instructions.

*Note* Many of the variables are already placed and set to None. This is because these variables are used in other parts of the code that have been provided to you. So do not change the name of the variable, but instead replace None with code specified by the instructions.



</br><details>
    <summary><b>(2.5)</b> Create an IECore object and use it to create IENetwork object with the provded model. Then get the input and output layer names. Use <code>ie_core</code> and <code>ie_net</code> as the variable names.</summary>

The paths for the model is provided. Do not change the variable name, `ie_core` and `ie_net` for ths file. The name of the input layer and output layer are stored in `inputs` and `outputs` dictionaries.

</details><br/>

<details><summary><b>(2.6)</b> Run <code>getCount()</code> on the request result to get the number of vehicles. </summary>

Use the request slot ID (`req_slot`) to get the result. Then get the number of vehicles from each inference request with `getCount()` function. remember that result of the inference itself can be accessed through the `outputs` attribute of the requests.

</details><br/>

<details>
    <summary><b>(2.7)</b> Start asynchronous processing on the next image. </summary>

Asynchronous (non-blocking) inference is started with `start_async()`.

</details><br/>

<details>

<summary><b>(2.8)</b> Handle the remaining requests. </summary>

The main while loop ends as soon as there are no more images to process, but there will be some inference that is still running. 
So we need to handle the remaining request.
We first wait until all requests are completed, then we can handle the remaining results in the queue.

This part is already implemented, so you just need to get the result.
This step should be identical to 2.6

</details><br/>

In [4]:
%%writefile main.py
import cv2
import sys
import queue
from openvino.inference_engine import IECore, IENetwork
from utils import *

device = sys.argv[1]

##! (2.5) Create IECore and IENetwork object from vehicle-detection-adas-0002 !##
xml_path="/data/intel/vehicle-detection-adas-0002/FP16/vehicle-detection-adas-0002.xml"
bin_path="/data/intel/vehicle-detection-adas-0002/FP16/vehicle-detection-adas-0002.bin"

ie_net = IENetwork(model=xml_path, weights=bin_path)

##! (2.5) Get the name of input and output layers. There is only one of each. !##
input_layer = next(iter(ie_net.inputs))
output_layer = next(iter(ie_net.outputs))

# Create ExecutableNetwork object using createExecNetwork in utils.py 
exec_net = createExecNetwork(ie_net, device)
print("ExecutableNetwork created with {} requests.".format(len(exec_net.requests)))

# Set the callback functions using setCallbackAndQueue() in utils.py 
c_queue = queue.Queue()
setCallbackAndQueue(exec_net, c_queue)

# Stats for processing
max_vehicles = 0
min_vehicles = 999      # this is safe as the max number of detectable objects is 200
sum_vehicles = 0
num_frames = 0
# Loading the data from a video
input_video = "/data/reference-sample-data/object-detection-python/cars_1900.mp4"
cap = cv2.VideoCapture(input_video)
while cap.isOpened():
    # Read the next frame
    ret, next_frame = cap.read()
    # Condition for the end of video
    if not ret:
        break
        
    ##! preprocess next_frame using prepImage from utils.py !##
    input_frame = prepImage(input_layer, next_frame, ie_net) 
    
    # using get to wait for the nextslot ID. Here we are setting a timeout of 30 seconds in case 
    #  there are issues with the callback and queue never gets populated. With timeout, this function
    #  will error out  with "Empty"

    req_slot, status = c_queue.get(timeout=30)
    
    if status == 0:
        ##! (2.6) Postprocess result from the request slot using getCount function from utils.py !##
        num_vehicles = getCount(exec_net.requests[req_slot].outputs[output_layer])
        
        max_vehicles = max(num_vehicles, max_vehicles)
        min_vehicles = min(num_vehicles, min_vehicles)
        sum_vehicles += num_vehicles
        num_frames += 1
        
    # Recall that None is what we set for the first time initializeation of queue, so we catch everything else.
    elif not status is None:
        print("There was error in processing an image")

    ##! (2.7) Start the next inference on the now open slot. !##
    exec_net.start_async(request_id=req_slot, inputs={input_layer:input_frame})

# Handle the remaining images.
#  first we wait for all request slots to complete
for req in exec_net.requests:
    req.wait()

# Handle remaining results 
while not c_queue.empty():
    req_slot, status = c_queue.get(timeout=30)
    
    if status == 0:
        ##! (2.8) Postprocess result from the request slot using getCount function from utils.py !##
        num_vehicles = getCount(exec_net.requests[req_slot].outputs[output_layer])
        max_vehicles = max(num_vehicles, max_vehicles)
        min_vehicles = min(num_vehicles, min_vehicles)
        sum_vehicles += num_vehicles
        
    # Recall that None is what we set for the first time initializeation of queue, so we catch everything else.
    elif not status is None:
        print("There was error in processing an image")
        
# Finally, reporting results.
print("Maximum number of cars detected: {}".format(max_vehicles))
print("Minimum number of cars detected: {}".format(min_vehicles))
print("average number of cars detected: {:.3g}".format(sum_vehicles/num_frames))

Overwriting main.py


### job file

Once again, the job file is provided for you. Note the if statement where we set up for FPGA if it is in the device list. Run the following cell to create the bash script `run.sh` to be used for benchmarking.

In [5]:
%%writefile run.sh

DEVICE=$1
source /opt/intel/openvino/bin/setupvars.sh

# Check if FPGA is used 
if grep -q FPGA <<<"$DEVICE"; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    export AOCL_BOARD_PACKAGE_ROOT=/opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2
    source /opt/altera/aocl-pro-rte/aclrte-linux64/init_opencl.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/2020-3_PL2_FP16_MobileNet_Clamp.aocx
    export CL_CONTEXT_COMPILER_MODE_INTELFPGA=3
fi
    
# Running the object detection code
python3 main.py $DEVICE

Overwriting run.sh


## Run the job

Finally, let us try to run the workload. 
Once again we've provided the same `submitToDevCloud` function.

**Note:** The toolkit is very verbose when using MYRIAD systems, so you may get a lot of additional output beyond what you are expecting. 


In [6]:
from devcloud_utils import submitToDevCloud
submitToDevCloud("run.sh", "CPU", script_args=["CPU"], files=["main.py","utils.py"])

Submitted. Job ID: FypmsVlL0BPihreCMFTEI97R9UjXN0Lh
Waiting for job to complete. This may take a few minutes............

[setupvars.sh] OpenVINO environment initialized
main.py:13: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  ie_net = IENetwork(model=xml_path, weights=bin_path)
ExecutableNetwork created with 1 requests.
Maximum number of cars detected: 8
Minimum number of cars detected: 1
average number of cars detected: 4.12



In [7]:
from devcloud_utils import submitToDevCloud
submitToDevCloud("run.sh", "GPU",  script_args=["GPU"], files=["main.py","utils.py"])

Submitted. Job ID: 3PVWp8SsGoHnSSVazKZUFIgBFT1sfcvR
Waiting for job to complete. This may take a few minutes..................

[setupvars.sh] OpenVINO environment initialized
main.py:13: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  ie_net = IENetwork(model=xml_path, weights=bin_path)
ExecutableNetwork created with 2 requests.
Maximum number of cars detected: 8
Minimum number of cars detected: 1
average number of cars detected: 4.13



In [8]:
from devcloud_utils import submitToDevCloud
submitToDevCloud("run.sh", "FPGA", script_args=["HETERO:FPGA,CPU"], files=["main.py","utils.py"])

Submitted. Job ID: MuLrAge4S7rSCkq0yulcFvnbJLlJvwe2
Waiting for job to complete. This may take a few minutes..........

[setupvars.sh] OpenVINO environment initialized
INTELFPGAOCLSDKROOT is not set
Using script's current directory (/opt/altera/aocl-pro-rte/aclrte-linux64)

aoc was not found, but aocl was found. Assuming only RTE is installed.

AOCL_BOARD_PACKAGE_ROOT is set to /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2. Using that.
Adding /opt/altera/aocl-pro-rte/aclrte-linux64/bin to PATH
Adding /opt/altera/aocl-pro-rte/aclrte-linux64/linux64/lib to LD_LIBRARY_PATH
Adding /opt/altera/aocl-pro-rte/aclrte-linux64/host/linux64/lib to LD_LIBRARY_PATH
Adding /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2/linux64/lib to LD_LIBRARY_PATH
aocl program: Running program from /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2/linux64/libexec
Programming device: a10gx_2ddr : Intel Vision Accelera

In [9]:
from devcloud_utils import submitToDevCloud
submitToDevCloud("run.sh", "VPU", script_args=["MYRIAD"], files=["main.py","utils.py"])

Submitted. Job ID: ZgQ3lAq7TTFjZk0354KJ6RyNKlcdTqwI
Waiting for job to complete. This may take a few minutes.....................

[setupvars.sh] OpenVINO environment initialized
main.py:13: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  ie_net = IENetwork(model=xml_path, weights=bin_path)
ExecutableNetwork created with 4 requests.
Maximum number of cars detected: 8
Minimum number of cars detected: 1
average number of cars detected: 4.11



Congratulations! You have just run multiple requests in parallel. 
From the output, you can see that multiple requests are being ran in parallel. 

**The final average vehicles detected to the third decimal will be asked in the quiz.**